# `pfynder` devel: v1

14.03.2021

**`pfynder`**

0. inputs: module name and function name to search for
1. find local of current python; effectively want `/usr/local/lib/pythonX.X/` where "pythonX.X" is the current version of the notebook
2. find the correct dir with the package's name, _ie_ "sklearn"
3. search through all `.py`-scripts using grep to find the function name
    - **Note** it could be in multiple modules, , filter to get those with "def"
4. output the results

In [1]:
!find /usr/local/lib/ -name "*sklearn" -type d | grep -e "python3.6"

/usr/local/lib/python3.6/dist-packages/sklearn


In [2]:
!whereis find

find: /usr/bin/find /usr/share/man/man1/find.1.gz /usr/share/info/find.info.gz


! ls /bin/find

In [3]:
import os

In [4]:
from subprocess import *

In [5]:
import platform

Get the current Python version (should be the kernel currently being used)

In [6]:
print(platform.python_version())
print(".".join(platform.python_version_tuple()[:2]))

3.6.9
3.6


In [7]:
import sys
print(sys.path)

['/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/home/sandm/.local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/home/sandm/.local/lib/python3.6/site-packages/IPython/extensions', '/home/sandm/.ipython']


In [8]:
def check_path(path):
    return (path != "") & \
        ((".zip" not in path) & \
         (".ipython" not in path) & \
         ("extensions" not in path)) & \
         ('packages' in path)

In [9]:
print("Paths to check")
checks = lambda p: f'\t{str(check_path(p)):5s}:' if check_path(p) else f'\t\t{str(check_path(p)):5s}:'
for p in sys.path:
    print(checks(p),p)

Paths to check
		False: /usr/lib/python36.zip
		False: /usr/lib/python3.6
		False: /usr/lib/python3.6/lib-dynload
		False: 
	True : /home/sandm/.local/lib/python3.6/site-packages
	True : /usr/local/lib/python3.6/dist-packages
	True : /usr/lib/python3/dist-packages
		False: /home/sandm/.local/lib/python3.6/site-packages/IPython/extensions
		False: /home/sandm/.ipython


In [10]:
!find /usr/local/lib/python3.6/dist-packages -name matplotlib -type d

/usr/local/lib/python3.6/dist-packages/matplotlib


In [11]:
def get_numb(numb, numbs):
    
    for n in numbs[::-1]:
        if numb < n: return n
    
    return numbs[-1]

def check_meth(dir_to_search, modulename):
    
    #print(l)
    module = modulename.split('.py:')[0]
    func = modulename.split('def ')[1].split("(")[0]

    location = f"{dir_to_search}/{module}"+".py"

    # Check for all class names and their line numbers
    p1 = Popen(["grep", "-n", "^class", location], stdout=PIPE)
    output = p1.communicate()[0]                

    classdic = [s.split(':class ') for s in output.decode("utf-8").split('\n') if s != '']
    classdic = {int(pp[0]):pp[-1].split("(")[0] for pp in classdic}                    

    #print(classdic)
    classkeys = sorted(classdic.keys())

    p1 = Popen(["grep", "-n", "^    def", location], stdout=PIPE)
    output = p1.communicate()[0]                

    methdic = [s.split(':    def ') for s in output.decode("utf-8").split('\n') if func in s]
    methdic = {int(pp[0]):pp[-1].split("(")[0] for pp in methdic}
    methkeys = sorted(methdic.keys())
    #print(methdic)

    module += "."+classdic[[get_numb(kk, classkeys) for kk in methkeys ][0]]
    module = module.replace('/','.')
    
    return module, func

In [15]:
def pfynd(package_to_check, item_to_find, ismodule=False):
    
    pyvers = ".".join(platform.python_version_tuple()[:2])
    
    paths = [p for p in sys.path if check_path(p)]
    
    #print(paths)
    
    not_found = []
    
    for path in paths:
        p1 = Popen(["find", path, "-type", "d", "-name", package_to_check], stdout=PIPE)
        p2 = Popen(["grep", f"python{pyvers}"], stdin=p1.stdout, stdout=PIPE)
        p1.stdout.close()  # Allow p1 to receive a SIGPIPE if p2 exits.
        output = p2.communicate()[0]

        dirs_to_search = [s for s in output.decode("utf-8").split('\n') if s != '']

        if dirs_to_search == []:
            not_found.append(f"\n\t\'{package_to_check}\' not found in {path}\n")
            continue

        #print(dirs_to_search)
        
        dir_to_search = dirs_to_search[0]

        #print(dir_to_search)

        if ismodule:
            p1 = Popen(["find", dir_to_search, "-type", "d", "-name", item_to_find], stdout=PIPE)
            #p3 = Popen(["grep", "def "], stdin=p2.stdout, stdout=PIPE)
            #p1.stdout.close()  # Allow p1 to receive a SIGPIPE if p2 exits.
            output = p1.communicate()[0]
            
            locations = [s.split("/"+package_to_check+"/")[1] for s in output.decode("utf-8").split('\n') if s != '']
            
            if len(locations) != 0:
                
                print(f"{package_to_check}")

                for l in locations:

                    print(f"\t{l}")

        else:
            p1 = Popen(["find", dir_to_search, "-type", "f", "-name", "*.py"], stdout=PIPE)
            p2 = Popen(["xargs", "grep", item_to_find], stdin=p1.stdout, stdout=PIPE)
            p3 = Popen(["grep", "def "], stdin=p2.stdout, stdout=PIPE)
            p1.stdout.close()  # Allow p1 to receive a SIGPIPE if p2 exits.
            p2.stdout.close()  # Allow p1 to receive a SIGPIPE if p2 exits.
            output = p3.communicate()[0]
            
            #print(output)

            locations = [s.split("/"+package_to_check+"/")[1] for s in output.decode("utf-8").split('\n') if s != '']

            #print("locations:\n\t", '\n\t'.join(locations))

            dc = {}
            for l in locations:
                
                #check if its a method, and then to which class it belongs
                if ('    ' in l) | ('\t' in l): 
                    
                    module, function = check_meth(dir_to_search, l)
                    
                    #print(module)
                    #try:
                    #    modpath = str_to_obj(package_to_check+'.'+module)
                    #    print(modpath)
                    #    print(isinstance(modpath, types.ClassType), hasattr(modpath, function))
                    #except:
                    #    pass
                    #         
                    
                else:
                    module = l.split('.py:')[0]
                    function = l.split('def ')[1].split("(")[0]                    
                
                if module in dc.keys():
                    dc[module].append(function)
                else:
                    dc[module]= [function]

            if len(dc.keys()) != 0:

                print(f"{package_to_check}")

                for k,v in dc.items():

                    print(f"\t{k}:")

                    for vv in v:
                        print(f"\t\t{vv}")
    
    show_fail = False
    if show_fail:
        print(not_found)
        
    return None

In [16]:
pfynd("matplotlib", "xlim", False)

matplotlib
	axes._base._AxesBase:
		get_xlim
		set_xlim
	pyplot:
		xlim
	tests.test_skew.SkewXAxes:
		lower_xlim
		upper_xlim
	projections.geo.LambertAxes:
		set_xlim
